#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

In [0]:
%python
from pyspark.sql.session import SparkSession
print(spark)#already instantiated by databricks
spark1=SparkSession.builder.getOrCreate()
print(spark1)#we instantiated

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
%python

for folder in [
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/",
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/",
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1",
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2"
]:
    dbutils.fs.mkdirs(folder)

Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):
a. Volume vs DBFS/FileStore
b. Why production teams prefer Volumes for regulated data

****** NEED TO WORK LATER *******

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
%python

customer_csv = """101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
"""

usage_tsv = """customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
"""

tower_logs_region1 = """event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
"""


##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
%python
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", customer_csv, True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", usage_tsv,True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv", tower_logs_region1,True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region2.csv", tower_logs_region1,True)

dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2")

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
%python

# 3.1 Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup
spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"],header=True,inferSchema=True,sep="|",pathGlobFilter="*.csv",recursiveFileLookup=True).show()

#3.2 Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")
spark.read.option("header","True").option("inferSchema","True").option("sep","|").option("pathGlobFilter","*.csv").option("recursiveFileLookup","True").format("CSV").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/").show()


******** Compare the outputs and understand when each should be used. ( Need To Work )******** 

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
%python

# Customer
custdf =spark.read.options(header="False",inferSchema="True",sep=",",pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
custdf.show()

# Usage
usagedf=  spark.read.options(header="True",inferSchema="True",sep="\t",pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
usagedf.show()


####  Write a note on What changed when we use header or inferSchema with true/false?

<b>header="True" </b> :  It will display the data with user given header <br>
<b>header="False" </b>: It will not display the data with user given header. But defaultly it will show the column name like c0, c1, c2 ....  <br>
<b>inferSchema="True" </b>: It will analyze/Scan the column data and it will show the data type of column.   <br>
<b>inferSchema="False" </b>: It consider and show all the column data type as string (default) <br><br>


#### How schema inference handled “abc” in age?
Age contains both integer and string value so it is considerign the age column as string 


##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
%python
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,TimestampType
#5.1
custdf.toDF("Id","Name","Age","Location","SimType").show()

#5.2
custom_schema = StructType([StructField("cust_id",IntegerType(),False),StructField("voi_mins",IntegerType(),True),StructField("data_mb",IntegerType(),True),StructField("sms_count",IntegerType(),True)])

custom_df1=spark.read.schema(custom_schema).options(header="True", sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
custom_df1.show()

custom_df1.printSchema()

#5.3
tower_schema = StructType([
    StructField("eve_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("tower_id", StringType(), True),
    StructField("signal_strength", IntegerType(), True),
    StructField("event_timestamp", TimestampType(), True)
])

tower_df1=spark.read.schema(tower_schema).options(header="True", sep="|" ,pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")
tower_df1.show()



## 6. More to come (stay motivated)....